Lambda School Data Science

*Unit 2, Sprint 2, Module 3*

---

# Cross-Validation


## Assignment
- [ ] [Review requirements for your portfolio project](https://lambdaschool.github.io/ds/unit2), then submit your dataset.
- [ ] Continue to participate in our Kaggle challenge. 
- [ ] Use scikit-learn for hyperparameter optimization with RandomizedSearchCV.
- [ ] Submit your predictions to our Kaggle competition. (Go to our Kaggle InClass competition webpage. Use the blue **Submit Predictions** button to upload your CSV file. Or you can use the Kaggle API to submit your predictions.)
- [ ] Commit your notebook to your fork of the GitHub repo.


You won't be able to just copy from the lesson notebook to this assignment.

- Because the lesson was ***regression***, but the assignment is ***classification.***
- Because the lesson used [TargetEncoder](https://contrib.scikit-learn.org/categorical-encoding/targetencoder.html), which doesn't work as-is for _multi-class_ classification.

So you will have to adapt the example, which is good real-world practice.

1. Use a model for classification, such as [RandomForestClassifier](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html)
2. Use hyperparameters that match the classifier, such as `randomforestclassifier__ ...`
3. Use a metric for classification, such as [`scoring='accuracy'`](https://scikit-learn.org/stable/modules/model_evaluation.html#common-cases-predefined-values)
4. If you’re doing a multi-class classification problem — such as whether a waterpump is functional, functional needs repair, or nonfunctional — then use a categorical encoding that works for multi-class classification, such as [OrdinalEncoder](https://contrib.scikit-learn.org/categorical-encoding/ordinal.html) (not [TargetEncoder](https://contrib.scikit-learn.org/categorical-encoding/targetencoder.html))



## Stretch Goals

### Reading
- Jake VanderPlas, [Python Data Science Handbook, Chapter 5.3](https://jakevdp.github.io/PythonDataScienceHandbook/05.03-hyperparameters-and-model-validation.html), Hyperparameters and Model Validation
- Jake VanderPlas, [Statistics for Hackers](https://speakerdeck.com/jakevdp/statistics-for-hackers?slide=107)
- Ron Zacharski, [A Programmer's Guide to Data Mining, Chapter 5](http://guidetodatamining.com/chapter5/), 10-fold cross validation
- Sebastian Raschka, [A Basic Pipeline and Grid Search Setup](https://github.com/rasbt/python-machine-learning-book/blob/master/code/bonus/svm_iris_pipeline_and_gridsearch.ipynb)
- Peter Worcester, [A Comparison of Grid Search and Randomized Search Using Scikit Learn](https://blog.usejournal.com/a-comparison-of-grid-search-and-randomized-search-using-scikit-learn-29823179bc85)

### Doing
- Add your own stretch goals!
- Try other [categorical encodings](https://contrib.scikit-learn.org/categorical-encoding/). See the previous assignment notebook for details.
- In additon to `RandomizedSearchCV`, scikit-learn has [`GridSearchCV`](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html). Another library called scikit-optimize has [`BayesSearchCV`](https://scikit-optimize.github.io/notebooks/sklearn-gridsearchcv-replacement.html). Experiment with these alternatives.
- _[Introduction to Machine Learning with Python](http://shop.oreilly.com/product/0636920030515.do)_ discusses options for "Grid-Searching Which Model To Use" in Chapter 6:

> You can even go further in combining GridSearchCV and Pipeline: it is also possible to search over the actual steps being performed in the pipeline (say whether to use StandardScaler or MinMaxScaler). This leads to an even bigger search space and should be considered carefully. Trying all possible solutions is usually not a viable machine learning strategy. However, here is an example comparing a RandomForestClassifier and an SVC ...

The example is shown in [the accompanying notebook](https://github.com/amueller/introduction_to_ml_with_python/blob/master/06-algorithm-chains-and-pipelines.ipynb), code cells 35-37. Could you apply this concept to your own pipelines?


### BONUS: Stacking!

Here's some code you can use to "stack" multiple submissions, which is another form of ensembling:

```python
import pandas as pd

# Filenames of your submissions you want to ensemble
files = ['submission-01.csv', 'submission-02.csv', 'submission-03.csv']

target = 'status_group'
submissions = (pd.read_csv(file)[[target]] for file in files)
ensemble = pd.concat(submissions, axis='columns')
majority_vote = ensemble.mode(axis='columns')[0]

sample_submission = pd.read_csv('sample_submission.csv')
submission = sample_submission.copy()
submission[target] = majority_vote
submission.to_csv('my-ultimate-ensemble-submission.csv', index=False)
```

In [1]:
%%capture
import sys
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
import category_encoders as ce 
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV, cross_val_score

# If you're on Colab:
if 'google.colab' in sys.modules:
    DATA_PATH = 'https://raw.githubusercontent.com/LambdaSchool/DS-Unit-2-Kaggle-Challenge/master/data/'
    !pip install category_encoders==2.*

# If you're working locally:
else:
    DATA_PATH = '../data/'

In [2]:
train = pd.merge(pd.read_csv(DATA_PATH+'waterpumps/train_features.csv'), 
                 pd.read_csv(DATA_PATH+'waterpumps/train_labels.csv'))
test = pd.read_csv(DATA_PATH+'waterpumps/test_features.csv')
sample_submission = pd.read_csv(DATA_PATH+'waterpumps/sample_submission.csv')

# Split train into train & val
train, val = train_test_split(train, train_size=0.80, test_size=0.20, 
                              stratify=train['status_group'], random_state=42)

train.shape, val.shape, test.shape

((47520, 41), (11880, 41), (14358, 40))

In [3]:
# clean the 3% of values that are in Null Island on lat long 

def wrangle(X):
    # Wrangle train, val, and test together
    
    # Prevent SettingWithCopyWarning
    X = X.copy()
    
   # fix null island
    X['latitude'] = X['latitude'].replace(-2e-08, 0)
    
    # Fix columns with zeros
    cols_with_zeros = ['longitude', 'latitude','construction_year', 
                       'gps_height', 'population']
    for col in cols_with_zeros:
        X[col] = X[col].replace(0, np.nan)

    # have lat and long for location
    X = X.drop(columns=['subvillage', 'region_code', 'ward'])

    # fix columns that seem to be reporting 0 as a arbitrary value
    X['funder'].replace(np.nan,0, inplace= True)
    X['installer'].replace(np.nan,0, inplace= True)
    X['permit'].replace(np.nan,0, inplace= True)
    X['scheme_management'].replace(np.nan,0, inplace= True)

    # Has True and False values
    X['public_meeting'].fillna(value=True, inplace=True)

    # Fixing values to values that seem to make more sense
    X['permit'].replace("0","TRUE", inplace= True) # Almost entirely not pay to use - consider as True
    X['public_meeting'].replace("0","TRUE", inplace= True)
    X['scheme_management'].replace(0, "Unknown", inplace=True)  #large number of Blanks - cannot fit this into None or Other group
    X['funder'].replace(0, "Other", inplace=True)
    X['installer'].replace(0, "Other", inplace=True)
    X['installer'].replace('-', "Other", inplace=True)
    X['construction_year'].replace(0, 2000, inplace=True) # Replacing 0 with median
    X['gps_height'].replace(0, 1166, inplace=True)        # Replacing 0 with median
    
    # duplicates dropped
    X = X.drop(columns=['quantity_group','extraction_type_group',
                        'extraction_type_class','waterpoint_type_group', 'payment_type',
                        'source_type'])

    # columns not very useful
    X = X.drop(columns=['num_private', 'recorded_by', 'id'])

    # Changing types 
    X['date_recorded'] = pd.to_datetime(X['date_recorded'])
    X['population'] = X['population'].astype('float64')
    X['gps_height'] = X['gps_height'].astype('float64')
    X['construction_year'] = X['construction_year'].astype('float64')
    X['amount_tsh'] = X['amount_tsh'].astype('float64')
    
    # split datetime up by mm, dd, yyyy
    X['month_recorded'] = X['date_recorded'].dt.month
    X['day_recorded'] = X['date_recorded'].dt.day
    X['year_recorded'] = X['date_recorded'].dt.year
    # no longer useful drop it
    X = X.drop(columns='date_recorded')

    # Feature engineering
    X['years'] = X['year_recorded'] - X['construction_year']
    X['years_MISSING'] = X['years'].isnull()
    
    return X


train = wrangle(train)
val = wrangle(val)
test = wrangle(test)

In [4]:
# Trying to perdict
target = 'status_group'

# don't let it leak out
train_features = train.drop(columns=[target])

# Numeric features
num_feats = train_features.select_dtypes(include='number').columns.tolist()

# Catagorical features
cat_feats = train_features.select_dtypes(exclude='number').columns.tolist()

# Get high cardinality <= 50
cardinality = train_features.select_dtypes(exclude='number').nunique()

low_card = cardinality[cardinality <= 50].index.tolist()

high_card = cardinality[cardinality >= 51].index.tolist()

features = num_feats + cat_feats

In [5]:
X_train = train[features]
y_train = train[target]
X_val = val[features]
y_val = val[target]
X_test = test[features]

In [ ]:
from scipy.stats import randint, uniform
from sklearn.preprocessing import LabelEncoder

pipeline = make_pipeline(
    ce.OneHotEncoder(use_cat_names=True),
    SimpleImputer(),
    RandomForestClassifier(n_jobs=-1, max_depth=21, random_state=42)
)

param = {
    'randomforestclassifier__n_estimators': randint(100, 500),
    'randomforestclassifier__max_depth': [18, 19, 20, 21, 22, None],
    'randomforestclassifier__max_features': uniform(0, 1),
    'simpleimputer__strategy': ['mean', 'median'],
}

# If you're on Colab, decrease n_iter & cv parameters
search = RandomizedSearchCV(
    pipeline, 
    param_distributions=param, 
    n_iter=10, 
    cv=3, 
    scoring='neg_mean_absolute_error', 
    verbose=10, 
    return_train_score=True, 
    n_jobs=-2
)

search.fit(X_train, y_train);

Fitting 3 folds for each of 10 candidates, totalling 30 fits


[Parallel(n_jobs=-2)]: Using backend LokyBackend with 3 concurrent workers.
[Parallel(n_jobs=-2)]: Done   2 tasks      | elapsed:  4.1min


In [ ]:
print('Best hyperparameters', search.best_params_)
print('Cross-validation MAE', -search.best_score_)

In [ ]:
pd.DataFrame(search.cv_results_).sort_values(by='rank_test_score').T